In [25]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [175]:
from order_placed.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value
from django.db.models import Avg, OuterRef, Subquery

In [176]:
df = read_frame(Betting.objects.filter(match="Srilanka vs India"))

In [177]:
df

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,7,fairbet,1000.0,Srilanka,BACK,1.85,Not declared yet,None,Srilanka vs India,2022-07-04 07:05:52.327891+00:00,2022-07-04 07:05:52.331877+00:00
1,8,fairbet,1000.0,Srilanka,LAY,1.85,Not declared yet,None,Srilanka vs India,2022-07-04 07:06:03.985142+00:00,2022-07-04 11:19:29.749531+00:00


In [122]:
# Client.objects.update(account_type=Case(
#     When(registered_on__lte=a_year_ago,
#          then=Value(Client.PLATINUM)),
#     When(registered_on__lte=a_month_ago,
#          then=Value(Client.GOLD)),
#     default=Value(Client.REGULAR)
# ),)

In [183]:
betting = Betting.objects.filter(match="Srilanka vs India").values('loss_profit')

In [188]:
bet = betting.annotate(
    betting_loss_profit=Case(
        When(
            status="BACK",
            then=F("amount")*0.8,
        ),
        default = F("loss_profit"),
        output_field = models.FloatField()
    )
)

In [203]:
Betting.objects.update(
    loss_profit=Case(
        When(status="B",
             match="Srilanka vs India",
             winning_team = "Not declared yet",
             then=F("amount")*F("odds") - F("amount")),
        When(status="L",
             match="Srilanka vs India",
             winning_team = "Not declared yet",
             then=F("amount")* -F("odds") + F("amount")),
        default=F("loss_profit")
)
     )

5